In [25]:
import requests
import pandas as pd

# Lista de códigos de indicadores para consultar
indicator_codes = ["AIR_12", "AIR_6","AIR_4", "AIR_11",
    "AIR_41", "UV_4","UV_2","WSH_2", "WSH_3", "WSH_4",
    "AIR_72","AIR_7"]

# Rango de años
years = "1990:2022"

# Códigos ISO de los países de América
america_countries = ["ARG","BRA","CAN","CHL","COL","CRI","CUB","DOM","ECU","SLV","GTM",
             "GUY","HTI","HND","JAM","MEX","NIC","PAN","PRY","PER","TTO","URY"]

# URL base de la API de la OMS
base_url = "https://ghoapi.azureedge.net/api"

# Diccionario para almacenar la correspondencia entre códigos de indicadores y sus nombres
indicador_nombres = {}

# Obtener nombres de indicadores
for code in indicator_codes:
    indicator_url = f"{base_url}/Indicator?$filter=IndicatorCode eq '{code}'"
    response = requests.get(indicator_url)
    if response.status_code == 200:
        indicator_data = response.json().get('value', [])
        if indicator_data:
            indicador_nombres[code] = indicator_data[0]['IndicatorName']

# Obtener datos de la API
data_frames = []
for indicator_code in indicator_codes:
    for country_code in america_countries:
        params = {
            "filter": f"Year ge {years.split(':')[0]} and Year le {years.split(':')[1]} and CountryCode eq '{country_code}' and IndicatorCode eq '{indicator_code}'",
            "format": "json",
        }

        response = requests.get(f"{base_url}/{indicator_code}", params=params)

        if response.status_code == 200:
            data = response.json().get('value', [])
            if data:
                filtered_data = [item for item in data if item['SpatialDim'] == country_code]
                if filtered_data:
                    for item in filtered_data:
                        item['IndicatorName'] = indicador_nombres[indicator_code]
                    data_frames.extend(filtered_data)

# Convertir a DataFrame
df = pd.DataFrame(data_frames, columns=["SpatialDim", "IndicatorName",  'TimeDim', 'NumericValue'])

# Cambiar nombres de las columnas
df = df.rename(columns={'SpatialDim': 'País', 'IndicatorName': 'Indicador', 'TimeDim': 'Año', 'NumericValue': 'Valor'})

df

,País,Indicador,Año,Valor
0,ARG,Household air pollution attributable deaths in...,2019,2.30901
1,ARG,Household air pollution attributable deaths in...,2019,0.98960
2,ARG,Household air pollution attributable deaths in...,2019,1.31941
3,BRA,Household air pollution attributable deaths in...,2019,140.35747
4,BRA,Household air pollution attributable deaths in...,2019,59.68436
...,...,...,...,...
1557,URY,Ambient air pollution attributable DALYs,2019,6099.20431
1558,URY,Ambient air pollution attributable DALYs,2019,798.50936
1559,URY,Ambient air pollution attributable DALYs,2019,1652.45584
1560,URY,Ambient air pollution attributable DALYs,2019,10825.68699
